In [1]:
import pyaudiowpatch as pyaudio
import numpy as np

In [14]:
RATE    = 16000
CHUNK   = 256


p = pyaudio.PyAudio()
wasapi_info = p.get_host_api_info_by_type(pyaudio.paWASAPI)
default_speakers = p.get_device_info_by_index(wasapi_info["defaultOutputDevice"])

if not default_speakers["isLoopbackDevice"]:
    for loopback in p.get_loopback_device_info_generator():
        """
        Try to find loopback device with same name(and [Loopback suffix]).
        Unfortunately, this is the most adequate way at the moment.
        """
        if default_speakers["name"] in loopback["name"]:
            default_speakers = loopback
            break
    else:
        exit()
        
print(f"Recording from: ({default_speakers['index']}){default_speakers['name']}")



stream = p.open(format=pyaudio.paInt16, channels=default_speakers["maxInputChannels"], rate=int(default_speakers["defaultSampleRate"]), input=True, input_device_index=default_speakers["index"], frames_per_buffer=CHUNK)

for i in range(int(20*RATE/CHUNK)): #do this for 10 seconds
    # print stream data here
    data = np.fromstring(stream.read(CHUNK),dtype=np.int16)
    peak=np.average(np.abs(data))*2
    bars="#"*int(50*peak/2**16)
    print("%04d %05d %s"%(i,peak,bars), end="\r")

stream.stop_stream()
stream.close()
p.terminate()

Recording from: (17)Speakers (PRO X Wireless Gaming Headset) [Loopback]
0000 00191 
0001 00485 
0002 00224 
0003 00303 
0004 00640 
0005 00419 
0006 00509 
0007 00442 
0008 00280 
0009 00510 
0010 00201 
0011 00500 
0012 00325 
0013 00299 
0014 00486 
0015 00267 
0016 00536 
0017 00325 
0018 00278 
0019 00388 
0020 00358 
0021 00557 
0022 00303 
0023 00273 
0024 00306 
0025 00343 
0026 00311 
0027 00388 
0028 00223 
0029 00393 
0030 00364 
0031 00535 
0032 00402 
0033 00308 


C:\temp\ipykernel_17140\4088117765.py:29: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data = np.fromstring(stream.read(CHUNK),dtype=np.int16)


0034 00593 
0035 00235 
0036 00363 
0037 00485 
0038 00360 
0039 00565 
0040 00339 
0041 00271 
0042 00464 
0043 00463 
0044 00415 
0045 00286 
0046 00303 
0047 00413 
0048 00467 
0049 00352 
0050 00334 
0051 00239 
0052 00362 
0053 00418 
0054 00234 
0055 00371 
0056 00204 
0057 00285 
0058 00718 
0059 00859 
0060 00759 
0061 00562 
0062 01273 
0063 00730 
0064 00450 
0065 00552 
0066 00714 
0067 00482 
0068 00491 
0069 01017 
0070 01081 
0071 00518 
0072 00895 
0073 00449 
0074 00616 
0075 00403 
0076 00420 
0077 00405 
0078 00285 
0079 00508 
0080 00399 
0081 00536 
0082 00187 
0083 00392 
0084 00293 
0085 00183 
0086 00450 
0087 00453 
0088 00554 
0089 00576 
0090 00448 
0091 00467 
0092 00439 
0093 00312 
0094 00475 
0095 00372 
0096 00447 
0097 00344 
0098 00263 
0099 00267 
0100 00394 
0101 00410 
0102 00415 
0103 00303 
0104 00322 
0105 00280 
0106 00431 
0107 00395 
0108 00285 
0109 00323 
0110 00172 
0111 00308 
0112 00328 
0113 00128 
0114 00334 
0115 00127 
0116 00298 
0117